Fecha: Julio de 2025

Se busca hacer análisis de frecuencias en las columnas de opinión por año, mes y periódico. E implementar modelado de tópicos. Primer método LDA.

# Librerias

In [150]:
# Librerías
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
from spacy.lang.es.stop_words import STOP_WORDS
import spacy
import pyLDAvis
import pyLDAvis.lda_model
import json
import pickle
from datetime import datetime

In [151]:
# Ruta del directorio que contiene los archivos CSV
folder_path = r'C:\Users\karen\Documents\HumanidadesDigitales_git\BDD_Corpus\BDD_CSV'


In [152]:
dataframes = []

for archivo in os.listdir(folder_path):
    if archivo.endswith('.csv'):
        file_path = os.path.join(folder_path, archivo)        
        try:
            df = pd.read_csv(file_path, encoding='latin-1', sep=';')  
            dataframes.append(df)
        except Exception as e:
            print(f'Error al procesar {file_path}: {e}')


In [153]:
# Concatenar todos los DataFrames en uno solo
if dataframes:
    corpus_completo = pd.concat(dataframes, ignore_index=True)

corpus_completo.head()                                                                                                                              

,Diario,Autor,Fecha,Título,Texto,Vínculo
0,El Espectador,Gonzalo Hernández,1 de enero de 2018,Fajardo: para nada tibio,"La Coalición Colombia Partido Alianza Verde, ...",https://web.archive.org/web/20180102104221/htt...
1,El Espectador,Eduardo Barajas Sandoval,1 de enero de 2018,Macedonia de Norte,Las interpretaciones de la historia sirven com...,https://web.archive.org/web/20180102104221/htt...
2,El Espectador,Daniel Emilio Rojas Castro,1 de enero de 2018,El nacionalismo según Vargas Llosa,La semana pasada Mario Vargas Llosa publicó un...,https://web.archive.org/web/20180102104221/htt...
3,El Espectador,Reinaldo Spitaletta,1 de enero de 2018,"Tiempo sagrado, tiempo profano","Pudiera decirse, sin ser una verdad absoluta, ...",https://web.archive.org/web/20180102104221/htt...
4,El Espectador,Aura Lucía Mera,1 de enero de 2018,La rebelión de los bueyes,Lo mejor del encierro de Las Ventas fueron los...,https://web.archive.org/web/20180102104221/htt...


In [154]:
# Limpiar espacios en blanco al inicio y al final de las cadenas
corpus_completo = corpus_completo.apply(lambda x: x.str.strip() if x.dtype == "object" else x)


In [155]:
# Formatear fechas

meses = {
    'enero': '01',
    'febrero': '02',
    'marzo': '03',
    'abril': '04',
    'mayo': '05',
    'junio': '06',
    'julio': '07',
    'agosto': '08',
    'septiembre': '09',
    'octubre': '10',
    'noviembre': '11',
    'diciembre': '12'
}

def convertir_fecha(fecha_str):
    partes = fecha_str.split(' de ')
    dia = partes[0].zfill(2)  # Asegura 2 dígitos para días 1-9
    mes = meses[partes[1]]
    año = partes[2]
    return f"{dia}/{mes}/{año}"  # Formato DD/MM/AAAA

corpus_completo['Fecha_formateada'] = corpus_completo['Fecha'].apply(convertir_fecha)
corpus_completo['Fecha_formateada'] = pd.to_datetime(corpus_completo['Fecha_formateada'], format='%d/%m/%Y', errors='coerce')

# Descripción Corpus

In [156]:
# filas que tienen fecha inválida
fechas_invalidas = corpus_completo[corpus_completo['Fecha_formateada'].isna()]
fechas_invalidas

,Diario,Autor,Fecha,Título,Texto,Vínculo,Fecha_formateada


In [157]:
autores = corpus_completo['Autor'].unique().tolist()

In [187]:
len(autores)

577

In [159]:
autores = sorted(autores)

In [160]:
print(autores)

['Abdón Espinosa Valderrama', 'Abel Veiga Copo', 'Adolfo León Atehortúa Cruz', 'Adolfo Meisel Roca', 'Adolfo Zableh Durán', 'Adriana Cooper', 'Adriana La Rotta', 'Adriana Noreña', 'Alberto Donadio', 'Alberto Galán', 'Alberto López de Mesa', 'Aldo Civico', 'Alejandro Alvarado', 'Alejandro Daly', 'Alejandro Peláez', 'Alejandro Reyes Posada', 'Alejandro Riveros González', 'Alejandro Tagliavini', 'Alexánder Cambero', 'Alfonso Carvajal', 'Alfonso Cuéllar', 'Alfonso Gómez Méndez', 'Alfonso Llano Escobar', 'Alfonso Sánchez Cadavid', 'Alfredo Molano Jimeno', 'Alister Ramírez Márquez', 'Allison Benson Hernandez', 'Allison Benson Hernández', 'Alonso Sánchez', 'Alpher Rojas C.', 'Alvaro Forero Tascón', 'Alvaro Jimenez', 'Amadeo Rodríguez Castilla', 'Ana Cristina Restrepo Jiménez', 'Ana Güezmes García', 'Ana Lucía Lenis', 'Ana Margarita González', 'Ana María Córdoba Barahona', 'Ana María Ramírez', 'Ana María Ruiz Perea', 'Ana Milena Muñoz de Gaviria', 'Andrea Aldana', 'Andrea Padilla Villarraga', 

In [184]:
# Contar filas 
len(corpus_completo[(corpus_completo['Fecha_formateada'] < pd.to_datetime('2019-01-01'))])


len(corpus_completo.query("Fecha_formateada < '2021-01-01' and Fecha_formateada > '2019-12-31' and Diario == 'El Tiempo'"))
len(corpus_completo.query("Fecha_formateada < '2020-01-01' and Fecha_formateada > '2018-12-31'"))




3666

13342

In [146]:
# Contar palabras totales en la columna 'Texto'
num_palabras = corpus_completo['Texto'].apply(lambda x: len(str(x).split()))

In [15]:
num_palabras.sum()

8377770

In [31]:
# Contar palabras distintas en la columna 'Texto'
texto_unido = ' '.join(corpus_completo['Texto'].dropna().astype(str).tolist())
palabras_distintas = set(texto_unido.split())

In [32]:
len(palabras_distintas)

336045

In [10]:
num_palabras.describe()

count    13344.000000
mean       627.830486
std        210.698741
min        128.000000
25%        523.000000
50%        590.000000
75%        682.000000
max       5305.000000
Name: Texto, dtype: float64

# Preprocesamiento

https://spacy.io/models/es

In [79]:
!python -m spacy download es_core_news_md

     ---------------------------------------- 0.0/42.3 MB ? eta -:--:--
     ---- ----------------------------------- 4.7/42.3 MB 35.7 MB/s eta 0:00:02
     ------------------ -------------------- 20.4/42.3 MB 61.5 MB/s eta 0:00:01
     --------------------------------------  41.4/42.3 MB 77.5 MB/s eta 0:00:01
     --------------------------------------- 42.3/42.3 MB 69.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')


In [55]:
# Preprocesamiento de los datos

# Cargar modelo de lenguaje en español de Spacy, el modelo más pequeño
#nlp = spacy.load("es_core_news_sm")

# Cargar modelo de lenguaje en español de Spacy, el modelo mediano
nlp = spacy.load("es_core_news_md")


# Modelo que incluye transformers 
#nlp = spacy.load("es_dep_news_trf")

## Función de preprocesamiento 

In [56]:
def preprocesar_texto(texto):
    # Limpieza
    texto = re.sub(r'http\S+|www\S+|https\S+', '', texto, flags=re.MULTILINE) # eliminar URLs
    texto = re.sub(r'\S+@\S+', '', texto) # eliminar correos electrónicos
    texto = re.sub(r'[^\w\sáéíóúñÁÉÍÓÚÑ]', ' ', texto) # mantener solo caracteres alfanuméricos y acentos
    
    # Procesamiento con spaCy tokenizacion
    doc = nlp(texto)
    
    # Lematización y filtrado
    tokens = [
        token.text for token in doc
        if not token.is_punct 
        and not token.is_stop
        and not token.is_space
        and token.pos_ in ['NOUN', 'VERB', 'PROPN','ADJ', 'NUM']  # Tomar solo sustantivos, verbos, nombres propios, adjetivos y numerales
        and len(token.lemma_) > 3  # Eliminar palabras muy cortas
        and token.text.isalpha()  # Solo palabras alfabéticas
    ]
    
    return " ".join(tokens)

# Títulos

In [67]:
titulos_procesados = corpus_completo['Título'].astype(str).apply(preprocesar_texto)

In [46]:
titulos_procesados

0        César Gaviria César Mondragón
1                  recomendaciones año
2                        Fajardo tibio
3                      Macedonia Norte
4            nacionalismo Vargas Llosa
                     ...              
13339          Capitalismo escandinava
13340        derrotar Guachos Amazonas
13341              Porte armas derecho
13342                         Congreso
13343            Casanare bicentenario
Name: Título, Length: 13344, dtype: object

In [68]:
corpus_completo['Título'][13339]

'Capitalismo a la escandinava'

In [29]:
# Vectorización (creación de matriz documento-término)
vectorizer = CountVectorizer(max_df=0.60, min_df=5, max_features=2000)
X = vectorizer.fit_transform(titulos_procesados)

In [48]:
tfidf_vectorizer = TfidfVectorizer(**vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(titulos_procesados)

In [49]:
X.shape

(13344, 1404)

In [50]:
dtm_tfidf.shape

(13344, 1355)

In [21]:
# Función para mostrar los tópicos
def mostrar_topicos(modelo, vectorizer, n_palabras=10):
    palabras = vectorizer.get_feature_names_out()
    for idx, topico in enumerate(modelo.components_):
        print(f"Tópico #{idx}:")
        print(", ".join([palabras[i] for i in topico.argsort()[:-n_palabras - 1:-1]]))
        print()

In [57]:
# Modelado de tópicos con LDA
lda = LatentDirichletAllocation(n_components=10, random_state=42)
lda.fit(X)

mostrar_topicos(lda, vectorizer)

Tópico #0:
presidente, justicia, voto, colombiano, ley, blanco, casa, realidad, semana, venir

Tópico #1:
duque, tola, maruja, derecho, social, venezuela, prima, niño, farc, presidente

Tópico #2:
trump, crisis, nacional, político, ojo, ciencia, democrático, paro, mundial, liderazgo

Tópico #3:
guerra, petro, político, defensa, duque, palabra, violencia, iván, arte, morir

Tópico #4:
año, país, tiempo, economía, coronavirus, volver, reforma, cambio, debate, jugar

Tópico #5:
colombia, política, historia, pandemia, corrupción, miedo, público, seguridad, universidad, corte

Tópico #6:
bogotá, fiscal, mujer, futuro, memoria, agua, pacto, fútbol, enemigo, salir

Tópico #7:
paz, mundo, gobierno, centro, covid, pasar, lección, hora, muerte, desarrollo

Tópico #8:
democracia, uribir, gazapera, elección, ganar, libro, tierra, malo, petro, fajardo

Tópico #9:
vida, educación, salud, matar, jep, presidencial, vuelta, viejo, empleo, santrich



In [51]:
# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=10, random_state=0)
lda_tfidf.fit(dtm_tfidf)

,n_components,10
,doc_topic_prior,None
,topic_word_prior,None
,learning_method,'batch'
,learning_decay,0.7
,learning_offset,10.0
,max_iter,10
,batch_size,128
,evaluate_every,-1
,total_samples,1000000.0
,perp_tol,0.1


In [58]:
mostrar_topicos(lda_tfidf, tfidf_vectorizer)

Tópico #0:
política, años, covid, tiempos, salud, arte, seguridad, agua, palabras, lecciones

Tópico #1:
paz, trump, historia, uribe, bogotá, voto, prima, mujeres, social, impuestos

Tópico #2:
petro, centro, economía, reforma, violencia, farc, niños, amor, semana, polarización

Tópico #3:
año, justicia, educación, miedo, futuro, odio, problema, cultura, china, señor

Tópico #4:
país, corrupción, defensa, memoria, muerte, claudia, congreso, electoral, preguntas, consulta

Tópico #5:
gobierno, crisis, tiempo, populismo, fútbol, votar, fajardo, libertad, palabra, vargas

Tópico #6:
guerra, presidente, mundo, pandemia, hora, derechos, mundial, paro, maduro, realidad

Tópico #7:
gazapera, fiscal, virus, gracias, desarrollo, plan, derecha, cuidado, allá, cuarentena

Tópico #8:
duque, colombia, vida, democracia, santos, iván, ley, pacto, presidente, sociales

Tópico #9:
venezuela, elecciones, coronavirus, tola, maruja, derecho, jep, esperanza, tierra, ojo



In [63]:
# Vectorización incluyendo bigrams
vectorizer_bigrams = TfidfVectorizer(ngram_range=(1, 5), max_df=0.90, min_df=2)
X_bigrams = vectorizer_bigrams.fit_transform(titulos_procesados)

# LDA con bigrams
lda_bigrams = LatentDirichletAllocation(n_components=5, random_state=42)
lda_bigrams.fit(X_bigrams)

mostrar_topicos(lda_bigrams, vectorizer_bigrams)

Tópico #0:
paz, bogotá, año, economía, corrupción, fiscal, tiempos, uribe, virus, trump

Tópico #1:
gazapera, duque, guerra, años, petro, elecciones, justicia, memoria, política, futuro

Tópico #2:
democracia, tiempo, crisis, muerte, agua, jep, odio, coronavirus, semana, derechos

Tópico #3:
presidente, venezuela, gobierno, mundo, duque, covid, defensa, hora, iván, violencia

Tópico #4:
país, colombia, historia, vida, pandemia, miedo, mujeres, derecho, centro, claudia



In [20]:
#Visualize and analyze reuslts
import pyLDAvis
import pyLDAvis.lda_model
pyLDAvis.enable_notebook() # For displaying in Jupyter notebooks



In [64]:
pyLDAvis.lda_model.prepare(lda_bigrams, X_bigrams, vectorizer_bigrams)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.133966 -0.143445       1        1  20.570638
4     -0.192587 -0.122437       2        1  20.017999
0     -0.060338  0.135732       3        1  19.947386
3      0.110822  0.026953       4        1  19.763418
2      0.008138  0.103197       5        1  19.700558, topic_info=             Term       Freq      Total Category  logprob  loglift
2138     gazapera  68.000000  68.000000  Default  30.0000  30.0000
3964   presidente  52.000000  52.000000  Default  29.0000  29.0000
1567        duque  84.000000  84.000000  Default  28.0000  28.0000
3667         país  49.000000  49.000000  Default  27.0000  27.0000
2252       guerra  43.000000  43.000000  Default  26.0000  26.0000
...           ...        ...        ...      ...      ...      ...
4871         tola  10.684830  23.456506   Topic5  -5.7748   0.8382
3047       maruja  10.411496  23.224400   Topic5  -5.8007   0.8222
3661          paz  12.638750  70.731854   Topic5  -5.6068  -0.0976
4872  tola maruja  10.070574  22.931044   Topic5  -5.8340   0.8016
3877     política  11.114039  58.507885   Topic5  -5.7354  -0.0364

[245 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
11        3  0.940738    aborto
94        5  0.976921      agua
195       2  0.219773      amor
195       4  0.109887      amor
195       5  0.659319      amor
...     ...       ...       ...
5246      1  0.770576      voto
5246      5  0.220165      voto
5254      2  0.958423    vuelta
5268      1  0.977446  víctimas
5297      2  0.968326    álvaro

[287 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 1, 4, 3])

In [38]:
pyLDAvis.lda_model.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.231420 -0.229889       1        1  10.848092
5     -0.113972 -0.022858       2        1  10.721805
0      0.100154  0.176266       3        1  10.255510
1     -0.158537 -0.128538       4        1  10.179660
7      0.201470 -0.025822       5        1  10.146594
6      0.002906  0.114367       6        1   9.975868
2      0.017808  0.093220       7        1   9.945304
4     -0.123221 -0.151572       8        1   9.938658
8     -0.149681  0.010216       9        1   9.122998
3     -0.008346  0.164609      10        1   8.865510, topic_info=          Term        Freq       Total Category  logprob  loglift
405      duque  122.000000  122.000000  Default  30.0000  30.0000
92         año   96.000000   96.000000  Default  29.0000  29.0000
960        paz   97.000000   97.000000  Default  28.0000  28.0000
560   gazapera   78.000000   78.000000  Default  27.0000  27.0000
961       país   72.000000   72.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
784   martínez    7.379927    8.195176  Topic10  -5.1471   2.3182
638   impuesto   22.460204   27.553345  Topic10  -4.0341   2.2186
1028  pregunta   11.756833   18.917055  Topic10  -4.6815   1.9474
862    naranja   12.028694   19.895165  Topic10  -4.6586   1.9198
22        agro    8.377077   10.532888  Topic10  -5.0204   2.1940

[382 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1         7  0.905958  abierto
3         7  0.931172   aborto
7        10  0.941796   acabar
11        8  0.896919   acción
12       10  0.949338    acoso
...     ...       ...      ...
1382      5  0.082091    votar
1383      2  0.972489     voto
1385      5  0.958863   vuelta
1387      5  0.933409  víctima
1392      3  0.934514   álvaro

[422 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 6, 1, 2, 8, 7, 3, 5, 9, 4])

In [92]:
pyLDAvis.disable_notebook()

In [151]:
%pip install pyldavis


  Using cached pyLDAvis-3.4.1-py3-none-any.whl.metadata (4.2 kB)
Using cached pyLDAvis-3.4.1-py3-none-any.whl (2.6 MB)


# Texto Completo LDA

## Preprocesamiento y representación vectorial

In [7]:
# Para ver las stopwords en español
print(STOP_WORDS)

{'sí', 'tus', 'hago', 'qué', 'solos', 'tener', 'será', 'nuevo', 'podemos', 'son', 'estamos', 'queremos', 'va', 'podeis', 'eso', 'los', 'muchas', 'lleva', 'o', 'hicieron', 'qeu', 'tuyos', 'mediante', 'excepto', 'cuando', 'tenga', 'cualquier', 'este', 'diez', 'por', 'ésta', 'ésas', 'ellos', 'mismo', 'debe', 'deben', 'da', 'nada', 'está', 'cierto', 'bien', 'segundo', 'cuánta', 'nuestras', 'pasada', 'podrán', 'tendrán', 'peor', 'cuál', 'les', 'poco', 'parece', 'uno', 'tuvo', 'sin', 'expresó', 'nuestro', 'seis', 'aquel', 'tanto', 'saber', 'aquella', 'partir', 'tercera', 'podrían', 'así', 'ningunos', 'encuentra', 'habrá', 'consiguen', 'estaba', 'también', 'éstos', 'manera', 'podrá', 'usa', 'ahi', 'mucho', 'te', 'sólo', 'estas', 'tuya', 'soy', 'hacerlo', 'detras', 'sera', 'poner', 'mi', 'pues', 'tendrá', 'eran', 'fuimos', 'ahí', 'igual', 'existe', 'ésa', 'despacio', 'he', 'usas', 'cuáles', 'quedó', 'yo', 'ya', 'habia', 'todo', 'entre', 'respecto', 'fue', 'esa', 'primeros', 'dijo', 'siempre', 

In [57]:
# Añadir stopwords 
nlp.Defaults.stop_words |= {"tola","maruja","gazapera"}

In [65]:
# Procesar los primeros 1000 textos de columnas, eliminando urls, stopwords del diccionario de Spacy y lematizando
procesar = False
if procesar:
    columnas_procesadas = corpus_completo['Texto'].astype(str).apply(preprocesar_texto)


In [66]:
# Guardar en archivo JSON el texto procesado para evitar correrlo de nuevo
with open('columnas_procesadas.json', 'w', encoding='utf-8') as f:
    json.dump(columnas_procesadas.tolist(), f, ensure_ascii=False, indent=4)  

In [58]:
# Cargar el texto procesado desde el archivo JSON
with open('columnas_procesadas.json', 'r', encoding='utf-8') as f:
    columnas_procesadas = pd.Series(json.load(f))

In [59]:
longitudes = columnas_procesadas.apply(lambda x: len(x.split()))

print(longitudes.describe())

count    13344.000000
mean       252.886016
std         90.060471
min         53.000000
25%        209.000000
50%        237.000000
75%        275.000000
max       2259.000000
dtype: float64


In [60]:
def eliminar_bigrama(texto):
    return texto.replace("email protected", "")
columnas_procesadas = columnas_procesadas.apply(eliminar_bigrama)

Para la representación se puede matriz término documento por frecuencia y matriz con ponderación tf-idf, consideraré los bigramas también.

El parametro max_df indica ignorar palabras que aparecen en un porcentaje de documentos mayor al dado (ignorar palabras muy frecuentes). El min_df es la frecuencia mínima en documentos, para eliminar palabras muy raras.

In [61]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(2, 4), max_df=0.7, min_df=10, max_features=15000)
matriz_columnas = tfidf_vectorizer.fit_transform(columnas_procesadas)

In [62]:
matriz_columnas.shape

(13344, 12845)

## Modelado Tópicos LDA
https://nbviewer.org/github/bmabey/pyLDAvis/blob/master/notebooks/LDA%20model.ipynb#topic=0&lambda=1&term= Para visualizar

In [75]:
# Función para mostrar los tópicos
def mostrar_topicos(modelo, vectorizer, n_palabras=10):
    palabras = vectorizer.get_feature_names_out()
    for idx, topico in enumerate(modelo.components_):
        print(f"Tópico #{idx}:")
        print(", ".join([palabras[i] for i in topico.argsort()[:-n_palabras - 1:-1]]))
        print()

In [64]:
# Modelado de tópicos con LDA
lda = LatentDirichletAllocation(n_components=10, random_state=42)
lda.fit(matriz_columnas)

mostrar_topicos(lda, tfidf_vectorizer)

Tópico #0:
donald trump, casa blanca, presidente trump, guerra mundial, política exterior, presidente unidos, medios comunicación, reino unido, unión europea, corea norte

Tópico #1:
frutas verduras, santo domingo, real madrid, desarrollo personalidad, libre desarrollo, duras penas, libre desarrollo personalidad, teatro santo, teatro santo domingo, garcía lorca

Tópico #2:
iván duque, centro democrático, álvaro uribe, corte suprema, gustavo petro, vargas lleras, presidente duque, juan manuel, uribe vélez, juan manuel santos

Tópico #3:
medios comunicación, presidente duque, libertad expresión, dosis mínima, lleras restrepo, carlos lleras, juan pablo, libertad prensa, investigador dejusticia, dejusticia profesor

Tópico #4:
lópez obrador, memoria histórica, nacional memoria, centro nacional, centro nacional memoria, nacional memoria histórica, centro nacional memoria histórica, cambio climático, manuel lópez, manuel lópez obrador

Tópico #5:
líderes sociales, derechos humanos, conflicto

In [76]:
vis_data = pyLDAvis.lda_model.prepare(lda, matriz_columnas, tfidf_vectorizer)

# Guardar como HTML y abrir en navegador
pyLDAvis.save_html(vis_data, 'lda_visualization.html')
import webbrowser
webbrowser.open('lda_visualization.html')

True

## BERTopic

https://maartengr.github.io/BERTopic/index.html#common Documentación del BERTopic

https://huggingface.co/dccuchile/bert-base-spanish-wwm-uncased 

https://colab.research.google.com/drive/18arPPe50szvcCp_Y6xS56H2tY0m-RLqv?usp=sharing#scrollTo=VUNsYrCETNar  Colab con ejemplo de uso


In [ ]:
%pip install bertopic

In [ ]:
%pip install tf-keras

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 18.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



In [30]:
%pip install hf_xet

   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 2.7/2.7 MB 26.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [43]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [66]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import pipeline

In [67]:
sentence_model = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")

In [38]:
topic_model = BERTopic(language="spanish",
    top_n_words=10,          # Palabras por tópico
    min_topic_size=20,       # Tamaño mínimo del cluster
    nr_topics=50,        # Número de tópicos
    calculate_probabilities=True,
    vectorizer_model=TfidfVectorizer(stop_words=list(STOP_WORDS)),
    embedding_model= sentence_model 
    )

Tarda 11 minutos en correr.

In [39]:
texto_seleccionado = corpus_completo['Texto'].astype(str).tolist()
#texto_seleccionado = columnas_procesadas

CORRER = True
if CORRER:
    topics, probs = topic_model.fit_transform(texto_seleccionado)

In [44]:
# PRECAUCION AL CORRER ESTO
# Guardar el modelo entrenado y su información
with open('topics_bert.json', 'w', encoding='utf-8') as f:
    json.dump(topics, f, ensure_ascii=False, indent=4)
with open('probs_bert.json', 'w', encoding='utf-8') as f:
    json.dump(probs.tolist(), f, ensure_ascii=False, indent=4)
with open('topic_model.pkl', 'wb') as f:
    pickle.dump(topic_model, f)

In [68]:
# PRECAUCION AL CORRER ESTO
# Leer modelo, probabilidades y tópicos
with open('topics_bert.json', 'r', encoding='utf-8') as f:
    topics = json.load(f) 
with open('probs_bert.json', 'r', encoding='utf-8') as f:
    probs = json.load(f)
with open('topic_model.pkl', 'rb') as f:
    topic_model = pickle.load(f)

In [69]:
topic_model.get_topic_info()


,Topic,Count,Name,Representation,Representative_Docs
0,-1,7083,-1_país_colombia_gobierno_años,"[país, colombia, gobierno, años, política, pre...",[Me parece que el doctor Vargas Lleras se vuel...
1,0,1219,0_economía_colombia_empresas_gobierno,"[economía, colombia, empresas, gobierno, país,...",[La economía lleva cuatro años estancada. En e...
2,1,525,1_virus_coronavirus_pandemia_covid,"[virus, coronavirus, pandemia, covid, salud, 1...","[En un hecho inédito para la humanidad, por pr..."
3,2,358,2_vida_mundo_casa_años,"[vida, mundo, casa, años, tiempo, amor, dios, ...",[¿Son entonces las mujeres culpables del maltr...
4,3,317,3_educación_universidad_estudiantes_universidades,"[educación, universidad, estudiantes, universi...",[En el año 1993 el expresidente César Gaviria ...
5,4,291,4_colombia_presidente_gobierno_país,"[colombia, presidente, gobierno, país, polític...",[Debo empezar esta columna diciendo que no soy...
6,5,281,5_paz_farc_santrich_guerrilla,"[paz, farc, santrich, guerrilla, guerra, jep, ...",[El lunes el país vio atónito cómo el CTI capt...
7,6,235,6_fútbol_jugadores_gol_equipo,"[fútbol, jugadores, gol, equipo, goles, equipo...","[La economía devastada, las fuerzas de ocupaci..."
8,7,222,7_música_arte_musical_teatro,"[música, arte, musical, teatro, festival, arti...",[El Ibagué Festival es un nuevo espacio cultur...
9,8,208,8_venezuela_maduro_guaidó_venezolano,"[venezuela, maduro, guaidó, venezolano, chávez...",[Venezuela es tendencia en todos los medios na...


In [70]:
topic_model.get_topic(2)

[('vida', 0.008060417730403812),
 ('mundo', 0.0050532043486178424),
 ('casa', 0.004723487294607641),
 ('años', 0.004512426428302214),
 ('tiempo', 0.00434080702148459),
 ('amor', 0.004197519672360857),
 ('dios', 0.004037030287553795),
 ('cosas', 0.00395077303627431),
 ('gente', 0.003872026684822089),
 ('alguien', 0.0038632782375553607)]

In [71]:
topic_model.representative_docs_[2]

['¿Son entonces las mujeres culpables del maltrato por no denunciar? ¡No, señores!. Sin lugar a dudas el 100 por ciento de la culpa recae sobre nosotros, los hombres. Los hombres que las tenemos muertas del miedo viviendo en un país que ocupa el puesto 14 en el mundo entre los países con más víctimas de ese flagelo.\n\nFaltaban unos diez minutos para que mi reloj marcara las ocho de la noche. Días antes, me había ofrecido a cocinar un arroz de mariscos para la familia de mi novia en la cena del 31 de diciembre. Aun cuando aprendí esa receta de mi madre, y me la sé de memoria, quise bañarme y vestirme con tiempo para hacer lo mío sin afán, y lograr que el plato estuviera a la altura de las expectativas de mis suegros.\n\nSalí del cuarto ya vestido y perfumado para dirigirme a la cocina de la casa en donde estábamos hospedados. Apenas empezaba a caminar hacia allá, cuando los ladridos estruendosos de unos perros de la propiedad de al lado me hicieron detener la marcha pensando que algo e

In [72]:
import plotly.express as px

# Obtener información de tópicos
topic_info = topic_model.get_topic_info()

# Filtrar outliers (tópico -1)
topic_info_filtered = topic_info[topic_info['Topic'] != -1]

# Gráfico de barras interactivo
fig = px.bar(topic_info_filtered,
             x='Topic',
             y='Count',
             color='Name',
             hover_data=['Representation'],
             title='Distribución de Tópicos',
             labels={'Count': 'N° Columnas', 'Name': 'Palabras Clave'},
             height=600)
fig.update_layout(xaxis={'type': 'category'})
fig.show()

¿Cómo forzar que todas las columnas queden en un tópico?